In [2]:
import numpy as np
import pandas as pd
import locale
import time
#import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


locale.setlocale(locale.LC_TIME, "de_DE")

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199

# pd.describe_option('display')
#plt.close("all")

In [3]:
# Create color set bades on UR CD colors
urCdColors = [
    "rgb(156,0,75)", # HEIDENELKENROT
    "rgb(0,155,119)", # TÜRKISGRÜN
    "rgb(0,135,178)", # SPEKTRALBLAU
    "rgb(174,167,0)", # LÄRCHENNADELGRÜN
    "rgb(236,98,0)", # ORANGEROT
    "rgb(191,0,42)", # HEUCHERAROT
    "rgb(79,184,0)", # BLATTGRÜN
    "rgb(0,137,147)", # EISVOGELBLAU
    "rgb(0,85,106)", # CAPRIBLAU
    "rgb(205,211,15)", # URANGELB
    "rgb(236,188,0)" # VATIKANGELB
]

# Go table header colors
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

# Get Results

In [4]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')
print("Anzahl an kompletten Daten:", len(surveyData))

# Add extra fields for comparison
surveyData['duration-date-based'] = surveyData.apply(lambda row: pd.to_datetime(row['date-last'])-pd.to_datetime(row['date-start']), axis=1)
surveyData['duration-date-based-in-seconds'] = surveyData.apply(lambda row: row['duration-date-based'].total_seconds(), axis=1)
surveyData['duration-delta-all-vs-start-end'] = surveyData.apply(lambda row: row['duration-over-all']-row['duration-date-based-in-seconds'], axis=1)

# Build groups depending on what viz participant had to solve first

listFirstParticipants=surveyData[surveyData['list-question-number']==6]
chordFirstParticipants=surveyData[surveyData['chord-question-number']==6]
mapFirstParticipants=surveyData[surveyData['map-question-number']==6]

Anzahl an kompletten Daten: 203


## Socio-Demographic Data
### Age

In [5]:
df = surveyData[['age', 'highest-preferenced-viz']]

# Label strings
chartLabel = f"Altersverteilung, n={len(df)}"
categoryGroups = ["Alter gesamt", "nach Listen Präferenz", "nach Chord Präferenz", "nach Map Präferenz"]
yLabel = "Alter in Jahren"
fig = go.Figure()

# Age over all
fig.add_trace(go.Box(y=df['age'], name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Liste"]['age'], name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Chord"]['age'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Map"]['age'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Alter nach Präferenz",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()

# Table age
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [42, 32, 22, 20, 18],
      [33, 25.5, 22, 20, 18],
      [57, 36, 24, 21, 18],
      [43, 30, 22, 20, 18]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Gender

In [22]:
df = surveyData[['sex']]

print(surveyData[['sex']])

df.fillna('nicht angegeben', inplace=True)
df = df.groupby(['sex'], dropna=False)['sex'].count().reset_index(name='count')


# Label strings
chartLabel = f"Verteiltung nach Geschlecht, n={len(surveyData[['sex']])}"
categoryGroups = ["Männlich", "Keine Angabe", "Weiblich"]

# fig = px.pie(df, values='count',  names='sex',  title=chartLabel, labels=chartLabel)

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'])])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.show()

# Table sex
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Geschlecht</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", "<b>Gesamt</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [len(surveyData[surveyData['sex'] == 'männlich'])],
        [surveyData['sex'].isnull().sum()],
        [len(surveyData[surveyData['sex'] == 'weiblich'])],
        [len(surveyData[['sex']])]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

          sex
0    männlich
1    männlich
2    männlich
3    männlich
4    weiblich
5    männlich
6    männlich
7    weiblich
8    weiblich
9    männlich
10   männlich
11   männlich
12   männlich
13   männlich
14   männlich
15   weiblich
16   männlich
17   männlich
18   männlich
19   weiblich
20   männlich
21   männlich
22   männlich
23   männlich
24   männlich
25   männlich
26   männlich
27   männlich
28   weiblich
29   männlich
30   männlich
31   männlich
32   weiblich
33   männlich
34   männlich
35   männlich
36   männlich
37   männlich
38   männlich
39   männlich
40   männlich
41   männlich
42   männlich
43   männlich
44   männlich
45   männlich
46   weiblich
47   männlich
48   männlich
49   weiblich
50   männlich
51   männlich
52   männlich
53   weiblich
54   weiblich
55   männlich
56   männlich
57   weiblich
58   männlich
59   männlich
60   männlich
61        NaN
62   männlich
63   männlich
64   weiblich
65   männlich
66        NaN
67   männlich
68   männlich
69   männlich
70   w

/var/folders/s3/0k49_ks152z51hj36nht1qyr0000gn/T/ipykernel_24565/1901008988.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

